KEYS: 相较于 v20240619 版本，增加了以下几点：

1. 移仓换月
2. 参数搜索

In [ ]:
import dai
import numpy as np
import pandas as pd
from datetime import datetime
from bigmodule import M
from bigtrader.constant import OrderStatus, Direction, Offset

def initialize(context):
    # 策略参数
    context.short_grids = (0, 0)
    context.long_grids = (6000, 6500)
    context.grid_interval = context.data['grid_interval']
    context.order_qty = context.data['order_qty']
    context.close_interval = context.grid_interval

    # 策略变量
    context.symbol = 'SR409.CZC'    # 交易合约
    context.curr_grid = 0           # 当前网格线
    context.next_grid = 0           # 下个网格线
    context.grid_info = {}          # 已开仓网格信息
    context.order_flag = 0          # 交易方向：1-做多；-1-做空
    context.stats_info_sum = {}     # 每日统计（全部）
    context.stats_info = []         # 每日统计：开仓次数和平仓次数
    context.isprint = 0             # 日志打印
    context.save_stats = False      # 存储CSV统计信息

def before_trading_start(context, data):
    context.stats_info = [0, 0]     # 重置每日统计结果
    context.subscribe(context.symbol)

    # 每日盘前初始化策略变量
    if not context.grid_info:
        # 第一次则判断昨日收盘价
        last_close = data.history(context.symbol, ['close'], 1, '1d')['close'].values[0]
        if (last_close >= context.short_grids[0]) and (last_close <= context.short_grids[1]):
            context.curr_grid = context.short_grids[0]
            context.next_grid = context.curr_grid + context.grid_interval
            context.order_flag = -1
        elif (last_close >= context.long_grids[0]) and (last_close <= context.long_grids[1]):
            context.curr_grid = context.long_grids[1]
            context.next_grid = context.curr_grid - context.grid_interval
            context.order_flag = 1
    else:
        context.grid_info = {k: v for k, v in context.grid_info.items() if v != 0}
        grid_info = context.grid_info.copy()
        context.grid_info = dict(sorted(grid_info.items()))
        min_grid = min(context.grid_info)
        max_grid = max(context.grid_info)
        if (min_grid >= context.short_grids[0]) and (max_grid <= context.short_grids[1]):
            context.curr_grid = max_grid
            context.next_grid = context.curr_grid + context.grid_interval
            context.order_flag = -1
        elif (min_grid >= context.long_grids[0]) and (max_grid <= context.long_grids[1]):
            context.curr_grid = min_grid
            context.next_grid = context.curr_grid - context.grid_interval
            context.order_flag = 1
    
    msg = '【盘前初始化】{} 网格方向: {}, 当前网格: {}, 下个网格: {}, 已开仓网格: {}'.format(data.current_dt, context.order_flag, context.curr_grid, context.next_grid, context.grid_info)
    context.write_log(msg, stdout=context.isprint)

def send_order(context, direction: str, offset: str, price: float, volume: int):
    price = round(price, 1)
    send_status = None
    if (direction == Direction.LONG) and (offset == Offset.OPEN):          # 买开
        if volume != 0:
            send_status = context.buy_open(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid - context.grid_interval
        context.stats_info[0] += 1
    elif (direction == Direction.SHORT) and (offset == Offset.OPEN):       # 卖开
        if volume != 0:
            send_status = context.sell_open(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid + context.grid_interval
        context.stats_info[0] += 1
    elif (direction == Direction.LONG) and (offset == Offset.CLOSE or offset == Offset.CLOSETODAY):        # 买平
        if volume != 0:
            send_status = context.buy_close(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid + context.grid_interval
        context.stats_info[1] += 1
    elif (direction == Direction.SHORT) and (offset == Offset.CLOSE or offset == Offset.CLOSETODAY):       # 卖平
        if volume != 0:
            send_status = context.sell_close(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid - context.grid_interval
        context.stats_info[1] += 1
    msg = '【发单】下单回报: {}, 方向：{}, 开平: {}, 价格: {}, 数量: {}, 当前网格: {}, 下个网格: {}'.format(context.get_error_msg(send_status), direction, offset, price, volume, context.curr_grid, context.next_grid)
    context.write_log(msg, stdout=context.isprint)

    # 更新已开仓网格信息
    if offset == 1:
        context.grid_info[price] = 0

def handle_tick(context, tick):
    if ((tick.time_int > 205000000) & (tick.time_int < 205999999)) or ((tick.time_int > 85500000) & (tick.time_int < 85999999)):
        return
    # 做多逻辑
    if context.order_flag == 1:
        if tick.bid_price1 < context.next_grid:        # 做多开仓 - 买开
            # print(f"test: {tick.datetime}, {tick.ask_price1}, {tick.bid_price1}, {context.curr_grid}, {context.next_grid}, {context.grid_info}")
            if context.next_grid in context.grid_info:
                return
            msg = f'【触发买开】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(context, direction=Direction.LONG, offset=Offset.OPEN, price=context.next_grid, volume=context.order_qty)
        elif tick.ask_price1 > context.curr_grid + context.close_interval:       # 做多平仓 - 卖平
            if context.curr_grid not in context.grid_info:
                return
            msg = f'【触发卖平】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(
                context, direction=Direction.SHORT, offset=Offset.CLOSE,
                price=context.curr_grid + context.close_interval,
                volume=context.grid_info[context.curr_grid]
            )
    elif context.order_flag == -1:
        if tick.ask_price1 >= context.next_grid:       # 做空开仓 - 卖开
            if context.next_grid in context.grid_info:
                return
            msg = f'【触发卖开】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(context, direction=Direction.SHORT, offset=Offset.OPEN, price=context.next_grid, volume=context.order_qty)
        elif tick.bid_price1 < context.curr_grid - context.close_interval:       # 做空平仓 - 买平
            if context.curr_grid not in context.grid_info:
                return
            msg = f'【触发买平】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(
                context, direction=Direction.LONG, offset=Offset.CLOSE,
                price=context.curr_grid - context.close_interval,
                volume=context.grid_info[context.curr_grid]
            )

def handle_order(context, order):
    context.write_log('【委托回报】交易时间: {} {}, 标的: {}, 订单id: {}, 方向:{}, 开平: {}, 下单价格: {}, 下单数量: {}, 订单状态: {}, 成交数量: {}'.format(
        order.insert_date, order.order_time, order.symbol, order.order_id, order.direction, order.offset, order.order_price,
        order.order_qty, order.order_status, order.filled_qty
    ), stdout=context.isprint)

    # 撤单
    if order.order_status == OrderStatus.CANCELLED:
        if (order.offset == Offset.OPEN) and (context.grid_info[order.order_price] == 0):
            # 开仓网格还没成交则直接从已开仓网格信息中删除
            del context.grid_info[order.order_price]
        # 其他情况：开仓网格部分成交撤单 & 平仓撤单，则没有任何操作

    # 更新已开仓网格信息
    if order.offset == Offset.OPEN:
        context.grid_info[order.order_price] = order.filled_qty
    elif (order.offset == Offset.CLOSE) or (order.offset == Offset.CLOSETODAY):
        grid_price = order.order_price + context.close_interval if order.direction==Direction.LONG else order.order_price - context.close_interval
        if (grid_price in context.grid_info) and (order.filled_qty != 0):
            del context.grid_info[grid_price]
    
    # print('test:', context.grid_info)

def handle_trade(context, trade):
    context.write_log('【成交回报】交易时间: {} {}, 标的: {}, 订单id: {}, 方向:{}, 开平: {}, 成交价格: {}, 成交数量: {}'.format(
        trade.trade_date, trade.trade_time, trade.symbol, trade.trade_id, trade.direction, trade.offset, trade.filled_price, trade.filled_qty
    ), stdout=context.isprint)

def after_trading(context, data):
    context.stats_info_sum[data.current_dt.strftime('%Y-%m-%d')] = [context.order_qty, context.grid_interval, context.stats_info]
    context.write_log(f'【盘后统计】{data.current_dt} 当日已开仓网格: {context.grid_info}', stdout=context.isprint)
    context.write_log(f'【盘后统计】{data.current_dt} 当日成交次数: {context.stats_info_sum}', stdout=context.isprint)

    # 保存统计数据
    df = pd.DataFrame(context.stats_info_sum).T.reset_index()
    df.columns = ['date', 'order_qty', 'interval', 'trade_nums']
    df[['open_nums', 'close_nums']] = pd.DataFrame(df['trade_nums'].tolist(), index=df.index)
    df.drop('trade_nums', axis=1, inplace=True)
    if context.save_stats is True:
        df.to_csv(f'stats_{context.order_qty}_{context.grid_interval}.csv')

now = datetime.now()
grid_interval = 10
order_qty = int(200/grid_interval)
data = {
    'instruments': ['SR409.CZC'],
    'start_date': '2024-03-19 21:00:00',
    'end_date': '2024-06-20 15:00:00',
    'order_qty': order_qty,
    'grid_interval': grid_interval,
}

M.bigtrader.v20(
    data=data,
    start_date='',
    end_date='',
    initialize=initialize,
    before_trading_start=before_trading_start,
    handle_tick=handle_tick,
    handle_trade=handle_trade,
    handle_order=handle_order,
    after_trading=after_trading,
    capital_base=100000001,
    frequency='tick',
    product_type='期货',
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy='open',
    order_price_field_sell='open',
    benchmark='000300.SH',
    plot_charts=True,
    debug=True,
    backtest_only=False,
    m_cached=False
)
print(f'!!参数!! 下单数量: {order_qty}, 网格间距: {grid_interval}, 耗时: {datetime.now()-now}')

In [ ]:
rets = {}
for grid_interval in [5, 7, 8, 9, 10, 11, 12, 13, 14]:
    order_qty = int(200/grid_interval)
    df = pd.read_csv(f'stats_{order_qty}_{grid_interval}.csv', index_col=0)
    df['return'] = df['order_qty'] * df['interval'] * df['close_nums']
    rets[f'{grid_interval}'] = {
        '交易次数': df['close_nums'].sum() + df['open_nums'].sum(),
        '平仓次数': df['close_nums'].sum(),
        '平仓总收益': df['return'].sum()
    }
df = pd.DataFrame(rets).T
df['收益'] = df['平仓总收益'] / 85600
df['收益'] = df['收益'].apply(lambda x: round(x, 3))

import matplotlib.pyplot as plt

# 创建柱状图1
plt.bar(df.index, df['收益'])
plt.xlabel('interval')
plt.ylabel('returns')
plt.show()